![read_agent_teaser](https://read-agent.github.io/img/teaser.png)

In [ ]:
!wget https://github.com/nyu-mll/quality/raw/main/data/v1.0.1/QuALITY.v1.0.1.htmlstripped.dev
import re, time, datetime, json, string, copy

In [ ]:
# @title Using OpenAI GPT model (DO NOT run the next cell if using GPT)
!pip3 install openai
import openai

key = 'YOUR API KEY'  #@param {type: "string"}
gpt_client = openai.OpenAI(api_key=key)
model_type = 'gpt'

def query_gpt_model(
    prompt: str,
    lm: str = 'gpt-3.5-turbo-1106',
    temperature: float = 0.0,
    max_decode_steps: int = 512,
    seconds_to_reset_tokens: float = 30.0,
) -> str:
  while True:
    try:
      raw_response = gpt_client.chat.completions.with_raw_response.create(
        model=lm,
        max_tokens=max_decode_steps,
        temperature=temperature,
        messages=[
          {'role': 'user', 'content': prompt},
        ]
      )
      completion = raw_response.parse()
      return completion.choices[0].message.content
    except openai.RateLimitError as e:
      print(f'{datetime.datetime.now()}: query_gpt_model: RateLimitError {e.message}: {e}')
      time.sleep(seconds_to_reset_tokens)
    except openai.APIError as e:
      print(f'{datetime.datetime.now()}: query_gpt_model: APIError {e.message}: {e}')
      print(f'{datetime.datetime.now()}: query_gpt_model: Retrying after 5 seconds...')
      time.sleep(5)

In [ ]:
# @title Using Google Gemini model (DO NOT run this if using GPT)
!pip3 install -q -U google-generativeai
import google.generativeai as genai

key = 'YOUR API KEY'  #@param {type: "string"}

genai.configure(api_key=key)
model = genai.GenerativeModel('gemini-pro')
model_type = 'gemini'

def query_gemini_model(
    prompt: str,
    retries: int = 10,
) -> str:
  while True and retries > 0:
    try:
      response = model.generate_content(prompt)
      text_response = response.text.replace("**", "")
      return text_response
    except Exception as e:
      print(f'{datetime.datetime.now()}: query_gemini_model: Error: {e}')
      print(f'{datetime.datetime.now()}: query_gemini_model: Retrying after 5 seconds...')
      retries -= 1
      time.sleep(5)

In [ ]:
def query_model(prompt):
  if model_type == "gpt":
    return query_gpt_model(prompt)
  elif model_type == "gemini":
    return query_gemini_model(prompt)

In [ ]:
#@title Load a QuALITY example

# Fields that are straight text copies from raw example to processed example.
_ONE2ONE_FIELDS = (
    'article',
    'article_id',
    'set_unique_id',
    'writer_id',
    'source',
    'title',
    'topic',
    'url',
    'writer_id',
    'author',
)

quality_dev = []

with open('QuALITY.v1.0.1.htmlstripped.dev', 'r') as f:
  for line in f.readlines():
    j = json.loads(line)
    fields = {k: j[k] for k in _ONE2ONE_FIELDS}
    fields.update({
        'questions': [q['question'] for q in j['questions']],
        'question_ids': [q['question_unique_id'] for q in j['questions']],
        'difficults': [q['difficult'] for q in j['questions']],
        'options': [q['options'] for q in j['questions']],
    })

    fields.update({
        'gold_labels': [q['gold_label'] for q in j['questions']],
        'writer_labels': [q['writer_label'] for q in j['questions']],
      })

    quality_dev.append(fields)

example = quality_dev[13]

In [ ]:
#@title Helper functions

all_lowercase_letters = string.ascii_lowercase  # "abcd...xyz"
bracketed_lowercase_letters_set = set(
    [f"({l})" for l in all_lowercase_letters]
)  # {"(a)", ...}
bracketed_uppercase_letters_set = set(
    [f"({l.upper()})" for l in all_lowercase_letters]
)  # {"(a)", ...}

choices = ['(A)', '(B)', '(C)', '(D)']

def get_index_from_symbol(answer):
  """Get the index from the letter symbols A, B, C, D, to extract answer texts.

  Args:
    answer (str): the string of answer like "(B)".

  Returns:
    index (int): how far the given choice is from "a", like 1 for answer "(B)".
  """
  answer = str(answer).lower()
  # extract the choice letter from within bracket
  if answer in bracketed_lowercase_letters_set:
    answer = re.findall(r"\(.*?\)", answer)[0][1]
  index = ord(answer) - ord("a")
  return index

def count_words(text):
  """Simple word counting."""
  return len(text.split())

def quality_gutenberg_parser(raw_article):
  """Parse Gutenberg articles in the QuALITY dataset."""
  lines = []
  previous_line = None
  for i, line in enumerate(raw_article.split('\n')):
    line = line.strip()
    original_line = line
    if line == '':
      if previous_line == '':
        line = '\n'
      else:
        previous_line = original_line
        continue
    previous_line = original_line
    lines.append(line)
  return ' '.join(lines)

In [ ]:
#@title ReadAgent (1) Episode Pagination

prompt_pagination_template = """
You are given a passage that is taken from a larger text (article, book, ...) and some numbered labels between the paragraphs in the passage.
Numbered label are in angeled brackets. For example, if the label number is 19, it shows as <19> in text.
Please choose one label that it is natural to break reading.
Such point can be scene transition, end of a dialogue, end of an argument, narrative transition, etc.
Please answer the break point label and explain.
For example, if <57> is a good point to break, answer with \"Break point: <57>\n Because ...\"

Passage:

{0}
{1}
{2}

"""

def parse_pause_point(text):
  text = text.strip("Break point: ")
  if text[0] != '<':
    return None
  for i, c in enumerate(text):
    if c == '>':
      if text[1:i].isnumeric():
        return int(text[1:i])
      else:
        return None
  return None


def quality_pagination(example,
                       word_limit=600,
                       start_threshold=280,
                       max_retires=10,
                       verbose=True,
                       allow_fallback_to_last=True):
  article = example['article']
  title = example['title']
  print(f"[Pagination][Article {title}]")
  paragraphs = quality_gutenberg_parser(article).split('\n')

  i = 0
  pages = []
  while i < len(paragraphs):
    preceding = "" if i == 0 else "...\n" + '\n'.join(pages[-1])
    passage = [paragraphs[i]]
    wcount = count_words(paragraphs[i])
    j = i + 1
    while wcount < word_limit and j < len(paragraphs):
      wcount += count_words(paragraphs[j])
      if wcount >= start_threshold:
        passage.append(f"<{j}>")
      passage.append(paragraphs[j])
      j += 1
    passage.append(f"<{j}>")
    end_tag = "" if j == len(paragraphs) else paragraphs[j] + "\n..."

    pause_point = None
    if wcount < 350:
      pause_point = len(paragraphs)
    else:
      prompt = prompt_pagination_template.format(preceding, '\n'.join(passage), end_tag)
      response = query_model(prompt=prompt).strip()
      pause_point = parse_pause_point(response)
      if pause_point and (pause_point <= i or pause_point > j):
        print(f"prompt:\n{prompt},\nresponse:\n{response}\n")
        print(f"i:{i} j:{j} pause_point:{pause_point}")
        pause_point = None
      if pause_point is None:
        if allow_fallback_to_last:
          pause_point = j
        else:
          raise ValueError(f"prompt:\n{prompt},\nresponse:\n{response}\n")

    page = paragraphs[i:pause_point]
    pages.append(page)
    if verbose:
      print(f"Paragraph {i}-{pause_point-1}", page)
    i = pause_point
  print(f"[Pagination] Done with {len(pages)} pages")
  return pages

pages = quality_pagination(example)

In [ ]:
#@title ReadAgent (2) Memory Gisting

prompt_shorten_template = """
Please shorten the following passage.
Just give me a shortened version. DO NOT explain your reason.

Passage:
{}

"""

def quality_gisting(example, pages, word_limit=600, start_threshold=280, verbose=True):
  article = example['article']
  title = example['title']
  word_count = count_words(article)
  print(f"[Gisting][Article {title}], {word_count} words")

  shortened_pages = []
  for i, page in enumerate(pages):
    prompt = prompt_shorten_template.format('\n'.join(page))
    response = query_model(prompt)
    shortened_text = response.strip()
    shortened_pages.append(shortened_text)
    if verbose:
      print("[gist] page {}:".format(i), shortened_text, flush=True)
  shortened_article = '\n'.join(shortened_pages)
  gist_word_count = count_words(shortened_article)
  if verbose:
    print("Shortened article:\n", shortened_article, flush=True)
  output = copy.deepcopy(example)
  output.update({'title': title, 'word_count': word_count, 'gist_word_count': gist_word_count, 'shortened_pages': shortened_pages, 'pages': pages})
  if verbose:
    print(f"compression rate {round(100.0 - gist_word_count/word_count*100, 2)}% ({gist_word_count}/{word_count})")
  return output
example_with_gists = quality_gisting(example, pages)

In [ ]:
#@title ReadAgent (3) Look-Up

prompt_lookup_template = """
The following text is what you remembered from reading an article and a multiple choice question related to it.
You may read 1 to 6 page(s) of the article again to refresh your memory to prepare yourselve for the question.
Please respond with which page(s) you would like to read.
For example, if your only need to read Page 8, respond with \"I want to look up Page [8] to ...\";
if your would like to read Page 7 and 12, respond with \"I want to look up Page [7, 12] to ...\";
if your would like to read Page 2, 3, 7, 15 and 18, respond with \"I want to look up Page [2, 3, 7, 15, 18] to ...\".
if your would like to read Page 3, 4, 5, 12, 13 and 16, respond with \"I want to look up Page [3, 3, 4, 12, 13, 16] to ...\".
DO NOT select more pages if you don't need to.
DO NOT answer the question yet.

Text:
{}

Question:
{}
{}

Take a deep breath and tell me: Which page(s) would you like to read again?
"""

prompt_answer_template = """
Read the following article and answer a multiple choice question.
For example, if (C) is correct, answer with \"Answer: (C) ...\"

Article:
{}

Question:
{}
{}

"""

def quality_parallel_lookup(example, verbose=True):
  preprocessed_pages = example['pages']
  article = example['article']
  title = example['title']
  word_count = example['word_count']
  gist_word_count = example['gist_word_count']
  pages = example['pages']
  shortened_pages = example['shortened_pages']
  questions = example['questions']
  options = example['options']
  gold_labels = example['gold_labels']  # numerical [1, 2, 3, 4]

  print(f"[Look-Up][Article {title}] {word_count} words")

  model_choices = []
  lookup_page_ids = []

  shortened_pages_pidx = []
  for i, shortened_text in enumerate(shortened_pages):
    shortened_pages_pidx.append("<Page {}>\n".format(i) + shortened_text)
  shortened_article = '\n'.join(shortened_pages_pidx)

  expanded_gist_word_counts = []
  for i, label in enumerate(gold_labels):
    # only test the first question for demo
    if i != 1:
      continue
    q = questions[i]
    print("question: ", q)
    options_i = [f"{ol} {o}" for ol, o in zip(choices, options[i])]
    print("options: ", "\n".join(options_i))
    prompt_lookup = prompt_lookup_template.format(shortened_article, q, '\n'.join(options_i))

    page_ids = []

    response = query_model(prompt=prompt_lookup).strip()

    try: start = response.index('[')
    except ValueError: start = len(response)
    try: end = response.index(']')
    except ValueError: end = 0
    if start < end:
      page_ids_str = response[start+1:end].split(',')
      page_ids = []
      for p in page_ids_str:
        if p.strip().isnumeric():
          page_id = int(p)
          if page_id < 0 or page_id >= len(pages):
            print("Skip invalid page number: ", page_id, flush=True)
          else:
            page_ids.append(page_id)

    if verbose:
      print("Model chose to look up page {}".format(page_ids))

    # Memory expansion after look-up, replacing the target shortened page with the original page
    expanded_shortened_pages = shortened_pages[:]
    if len(page_ids) > 0:
      for page_id in page_ids:
        expanded_shortened_pages[page_id] = '\n'.join(pages[page_id])

    expanded_shortened_article = '\n'.join(expanded_shortened_pages)
    expanded_gist_word_count = count_words(expanded_shortened_article)
    if verbose:
      print("Expanded shortened article:\n", expanded_shortened_article, flush=True)
    prompt_answer = prompt_answer_template.format(expanded_shortened_article, q, '\n'.join(options_i))

    # If the response doesn't follow the template, retry
    model_choice = None
    response = query_model(prompt=prompt_answer)
    response = response.strip()
    for j, choice in enumerate(choices):
      if response.startswith(f"Answer: {choice}") or response.startswith(f"Answer: {choice[1]}"):
        model_choice = j+1
        break
    is_correct = 1 if model_choice == label else 0
    print(f"question: {q}")
    print(f"reference answer: {choices[label]}, model prediction: {choices[model_choice]}, is_correct: {is_correct}")
    print(f"compression rate {round(100.0 - gist_word_count/word_count*100, 2)}% ({gist_word_count}/{word_count})")
    print(f"compression rate after look-up {round(100.0 - expanded_gist_word_count/word_count*100, 2)}% ({expanded_gist_word_count}/{word_count})")

quality_parallel_lookup(example_with_gists)

#Prompts that we used in the paper

In the following we show the prompts that were used for the QuALTIY, QMSum, NarrativeQA datasets with the PaLM 2-L model. While there are slight differences in prompt design, most of these are not due to optimizing prompts for specific datasets but rather a results of that each author wrote the prompts independently.

In [ ]:
# @title The prompts we used for QuALITY with PaLM 2-L


# Pagination
pagination_prompt_template = """
You are given a passage that is taken from a larger text (article, book, ...) and some numbered labels between the paragraphs in the passage.
Numbered label are in angeled brackets. For example, if the label number is 19, it shows as <19> in text.
Please choose one label that it is natural to break reading.
Such point can be scene transition, end of a dialogue, end of an argument, narrative transition, etc.
Please answer the break point label and explain.
For example, if <57> is a good point to break, answer with \"Break point: <57>\n Because ...\"

Passage:

{passage_text}
{end_tag}

"""
# passage_text: a chunk of text.
# end_tag: a string, whose value is "" if the text is at the end of the article, and otherwise "\n...".



# Gisting
gisting_prompt_template = """
Please shorten the following passage.
Just give me a shortened version. DO NOT explain your reason.

Passage:
{page_text}

"""
# page_text: a page of text



# Parallel Look-up (ReadAgent-P, up to 5 pages)
parallel_lookup_prompt_template = """
The following text is what you remembered from reading an article and a multiple choice question related to it.
You may read 1 to 5 page(s) of the article again to refresh your memory to prepare yourselve for the question.
Please respond with which page(s) you would like to read again.
For example, if your would like to only read Page 8, respond with \"I want to look up Page [8] to ...\";
if your would like to read Page 7 and 12, respond with \"I want to look up Page [7, 12] to ...\";
if your would like to read Page 2, 3, 7, 15 and 18, respond with \"I want to look up Page [2, 3, 7, 15, 18] to ...\".
DO NOT select more pages if you don't need to.
DO NOT answer the question yet.

Text:
{concatenated_gists}

Question:
{question}
{options}

Take a deep breath and tell me: Which page(s) would you like to read again?
"""
# concatenated_gists: concatenated gists
# question: a question
# options: multiple-choice options



# Sequential Look-up (ReadAgent-S, up to 5 pages)
sequential_lookup_prompt_template = """
The following text is what you remember from reading an article, followed by a question about the article.
You may read multiple pages of the article again to refresh your memory and prepare to answer the question.
Each page that you re-read can significantly improve your chance of answering the question correctly.
Please specify a SINGLE page you would like to read again or say "STOP".
To read a page again, respond with "Page $PAGE_NUM", replacing $PAGE_NUM with the target page number.
You can only specify a SINGLE page in your response at this time.
DO NOT select more pages if you don't need to.
To stop, simply say "STOP".
DO NOT answer the question in your response.

Text:
{concatenated_gists}
End of text.

Pages re-read already (DO NOT ask to read them again):
{past_page_numbers}

Question:
{question}
{options}

Specify a SINGLE page to read again, or say STOP:
"""
# concatenated_gists: concatenated gists
# past_page_numbers: page numbers that have already been retrieved
# question: a question
# options: options



# Response/Answer
answer_prompt_template = """
Read the following article and answer a multiple choice question.
For example, if (C) is correct, answer with \"Answer: (C) ...\"

Article:
{concatenated_pages_and_gists}

Question:
{question}
{options}

"""
# concatenated_pages_and_gists: concatenated raw pages and gists
# question: a question
# options: options

In [ ]:
# @title The prompts we used for QMSum with PaLM 2-L


# Pagination
pagination_prompt_template = """
You are given a passage that is taken from a larger meeting transcript.
There are some numbered labels between the paragraphs (like <0>) in the passage.
Please choose one label at a natural transition in the passage.
For example, the label can be at the end of a dialogue, the end of an argument, a change in the topic being discussed, etc.
Please respond with the label and explain your choice.
For example, if <57> is a natural transition, answer with "Label: <57>\n Because ..."

Passage:

{preceding_text}
{passage_text}
{end_tag}

"""
# preceding_text: a fraction of previous context
# passage_text: a chunk of text.
# end_tag: a string, whose value is "" if the text is at the end of the article, and otherwise "\n...".



# Gisting
gisting_prompt_template = """
Please shorten the following passage.
Just give a shortened version. DO NOT explain your reasoning.

Passage:
{page_text}

"""
# page_text: a page of text



# Parallel Look-up (ReadAgent-P, up to 2 pages)
parallel_lookup_prompt_template = """
The following text is what you remember from reading a meeting transcript, followed by a question about the transcript.
You may read 1 or 2 pages of the transcript again to refresh your memory to prepare to answer the question.
Please respond with which page(s) you would like to read.
For example, if your would only like to read Page 8, respond with "I want to look up Page [8] ..."
If you would like to read Page 7 and 12, respond with "I want to look up Page [7, 12] ...".
Only select as many pages as you need, but no more than 2 pages.
Don't answer the question yet.

Text:
{text}
End of text.

Question:
{question}

Which page(s) would you like to look up?
"""
# concatenated_gists: Concatenated gists
# question: a question



# Sequential Look-up (ReadAgent-S)
sequential_lookup_prompt_template = """
The following text is what you remember from reading a meeting transcript, followed by a question about the transcript.
You may read multiple pages of the transcript again to refresh your memory and prepare to answer the question.
Each page that you re-read can significantly improve your chance of answering the question correctly.
Please specify a SINGLE page you would like to read again or say "STOP".
To read a page again, respond with "Page $PAGE_NUM", replacing $PAGE_NUM with the target page number.
You can only specify a SINGLE page in your response at this time.
DO NOT select more pages if you don't need to.
To stop, simply say "STOP".
DO NOT answer the question in your response.

Text:
{concatenated_gists}
End of text.

Pages re-read already (DO NOT ask to read them again):
{past_page_numbers}

Question:
{question}

Specify a SINGLE page to read again, or say STOP:
"""
# concatenated_gists: concatenated gists
# past_page_numbers: page numbers that have already been retrieved
# question: a question



# Response/Answer
answer_prompt_template = """
Read the question and text below and then answer the question.

Question:
{question}

Text:
{concatenated_pages_and_gists}
End of Text.

Answer the question based on the above passage and retrieved pages. Your answer should be short and concise.
"""
# question: a question
# concatenated_pages_and_gists: concatenated raw pages and gists

In [ ]:
# @title The prompts we used for NarrativeQA - Gutenburg with PaLM 2-L


# Pagination
pagination_prompt_template = """
You are given a passage that is taken from a larger text (article, book, ...) and some numbered labels between the paragraphs in the passage.
Numbered label are in angeled brackets. For example, if the label number is 19, it shows as <19> in text.
Please choose one label that marks a major section break point.
Such points can be the beginning/end of a book, beginning/end of a chapter, end of a content table, a scene transition, end of a dialogue, etc.
If a point is chosen for the beginning of a book/chapter/etc and there is a title of the new book/chapter/etc, the break point must be chosen at a position right before the section number and title, not after.

Please answer the break point label and explain.
For example, if <57> is a good point to break, answer with \"Breakpoint: <57> ...\"

Text:

{preceding_text}
{passage_text}
{end_tag}

"""
# preceding_text: a fraction of previous context
# passage_text: a chunk of text.
# end_tag: a string, whose value is "" if the text is at the end of the article, and otherwise "\n...".



# Gisting
gisting_prompt_template = """
Please shorten the following passage.
Just give me a shortened version. DO NOT explain your reason.

Passage:
{page_text}

"""
# page_text: a page of text



# Parallel Look-up (ReadAgent-P, up to 2 pages)
parallel_lookup_prompt_template = """
The following text is what you remembered from reading an article and a question related to it.
You may read 1 or 2 page(s) of the article again to refresh your memory to prepare yourselve for the question.
Please respond with which page(s) you would like to read in the order of importance, beginning with the most important page number.
For example, if your only need to read Page 8, respond with \"I want to look up Page [8] to ...\";
if your would like to read Page 12 and 7, respond with \"I want to look up Page [12, 7] to ...\";
DO NOT select more pages if you don't need to.
You don't need to answer the question yet.

Text:
{concatenated_gists}

Question:
{question}

"""
# concatenated_gists: Concatenated gists
# question: a question



# Sequential Look-up (ReadAgent-S)
sequential_lookup_prompt_template = """
The following text is what you remember from reading a meeting transcript, followed by a question about the transcript.
You may read multiple pages of the transcript again to refresh your memory and prepare to answer the question.
Each page that you re-read can significantly improve your chance of answering the question correctly.
Please specify a SINGLE page you would like to read again or say "STOP".
To read a page again, respond with "Page $PAGE_NUM", replacing $PAGE_NUM with the target page number.
You can only specify a SINGLE page in your response at this time.
DO NOT select more pages if you don't need to.
To stop, simply say "STOP".
DO NOT answer the question in your response.

Text:
{concatenated_gists}
End of text.

Pages re-read already (DO NOT ask to read them again):
{past_page_numbers}

Question:
{question}

Specify a SINGLE page to read again, or say STOP:
"""
# concatenated_gists: concatenated gists
# past_page_numbers: page numbers that have already been retrieved
# question: a question



# Response/Answer
answer_prompt_template = """
{concatenated_pages_and_gists}

Question:
{question}

Answer the question based on the above passage and retrieved pages. Your answer should be short and concise.
"""
# concatenated_pages_and_gists: concatenated raw pages and gists
# question: a question

In [ ]:
# @title The prompts we used for NarrativeQA - Movie Scripts with PaLM 2-L


# Pagination
pagination_prompt_template = """
You are given a movie script and some numbered labels between the lines in the script.
Numbered label are in angeled brackets.
Please choose one label that it is natural to break reading. The label should be between <{start}> and <{end}>.
Such point can be scene transition, end of a dialogue, end of an argument, narrative transition, etc.
The answer should end with "The break point is: <number>", where the break point number is between angeled brackets.

Script:

{passage_text}
"""
# passage_text: a chunk of text.



# Gisting
gisting_prompt_template = """
Please shorten the following passage. The shortened passage should be in 128 tokens. Please refer to people with their full names whenever possible.
Just give me a shortened version. DO NOT explain your reason. If there is no meaning information in the passage, output "I don't have enough information to shorten the passage."

Passage:
{page_text}

"""
# page_text: a page of text



# Parallel Look-up (ReadAgent-P, up to 2 pages)
parallel_lookup_prompt_template = """
The following text includes a summary of each page in a movie script, followed by a question about the script.

Summary:
{concatenated_gists}

Question:
{question}

Based on the summary of each page, you may read the full details of 1 to 2 page(s) to obtain more information to answer the question.
Please respond with which page(s) you would like to read.
For example, if you only need to read Page X_0, the answer should end with \"I want to look up Page [X_0]\";
if you would like to read Page X_0 and X_1, the answer should end with \"I want to look up Page [X_0, X_1]\".
X_i above is a page index between 0 and {end}. DO NOT select more pages if you don't need to.
You don't need to answer the question yet.
"""
# concatenated_gists: Concatenated gists
# question: a question



# Sequential Look-up (ReadAgent-S)
sequential_lookup_prompt_template = """
The following text includes a summary of each page in a movie script, followed by a question about the script, and the previous answer based on the summary and already re-read pages.

Summary:
{concatenated_gists}

Pages re-read already (DO NOT ask to read them again):
{past_page_numbers}

Question:
{question}

Previous Answer:
{previous_answer}

Based on the summary of each page, you may read the full details of multiple pages to obtain more information to answer the question.
To read a page again, respond with "I want to look up Page $PAGE_NUM", replacing $PAGE_NUM with the target page number.
PAGE_NUM is a page index between 0 and {end}, excluding {pages_reread}.
You can only specify a SINGLE page in your response at this time. The page should not be in the re-read pages.
To stop, simply say "STOP".
DO NOT answer the question in your response.
You don't need to answer the question yet.
"""
# concatenated_gists: concatenated gists
# past_page_numbers: (only after the first query) page numbers that have already been retrieved
# question: a question
# previous_answer: the previous answer given by the model with gists and previously retrieved raw pages



# Response/Answer
answer_prompt_template = """
{concatenated_pages_and_gists}

Question:
{question}

Answer the question based on the above passage and retrieved pages. Your answer should be short and concise.
"""
# concatenated_pages_and_gists: concatenated raw pages and gists
# question: a question